In [1]:
import tweepy
from tweepy import OAuthHandler

# Stopwords
import re
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize

search_hashtag = "Election -filter:retweets"
tweets_since = '2019-11-01'

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
# Authentication
consumer_key = #### there should be keys to access twitter that I don't want to make public
consumer_secret = #### there should be keys to access twitter that I don't want to make public
access_token = #### there should be keys to access twitter that I don't want to make public
access_secret = #### there should be keys to access twitter that I don't want to make public

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth)

In [12]:
num_of_tweets = 2000000
tweets = tweepy.Cursor(api.search,
                   q=search_hashtag,
                   lang="en",
                   wait_on_rate_limit=True,
                   since=tweets_since).items(num_of_tweets)

all_tweets = [tweet.text for tweet in tweets]

all_tweets[:5]

['@FaceTheNation @margbrennan @LindseyGrahamSC You’ve sunk so low to give this guy a platform. He is in thrall to the… https://t.co/2Sh4FttORv',
 'Coming up on @WMBBTV @ 6: \n\n🔹 A yard sale was held in Franklin County to benefit the family whose daughters died in… https://t.co/vEpLxMBIBc',
 '@MarshaBlackburn Since we are talking about what we think, I think if you commit an act of sedition like say taking… https://t.co/pm269m6R1R',
 "@ScottLinnen @Stonekettle Where I'm from there's laws regarding displaying candidate signage during election day. E… https://t.co/m5km9rmzCF",
 'Scott Morrison’s political career is over. At the upcoming federal election, all Australians who have lost their bu… https://t.co/L5WM5hz2yy']

In [13]:
#Remove urls from tweet
def remove_url(tweet):
    return " ".join(re.sub("([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "", tweet).split())

all_tweets_no_urls = [remove_url(tweet) for tweet in all_tweets]
all_tweets_no_urls[:5]

['FaceTheNation margbrennan LindseyGrahamSC Youve sunk so low to give this guy a platform He is in thrall to the',
 'Coming up on WMBBTV 6 A yard sale was held in Franklin County to benefit the family whose daughters died in',
 'MarshaBlackburn Since we are talking about what we think I think if you commit an act of sedition like say taking',
 'ScottLinnen Stonekettle Where Im from theres laws regarding displaying candidate signage during election day E',
 'Scott Morrisons political career is over At the upcoming federal election all Australians who have lost their bu']

In [14]:
#Remove stopwords
marks_list = [',', '.', ';', '-', ':']
all_stopwords = stopwords.words('english')
all_stopwords.extend(marks_list)

def remove_stopwords(tweet):
  text_tokens = word_tokenize(tweet)
  return [word for word in text_tokens if not word in all_stopwords]

all_tweets_no_urls_no_stopwords = [remove_stopwords(tweet) for tweet in all_tweets_no_urls]
all_tweets_no_urls_no_stopwords[:5]

[['FaceTheNation',
  'margbrennan',
  'LindseyGrahamSC',
  'Youve',
  'sunk',
  'low',
  'give',
  'guy',
  'platform',
  'He',
  'thrall'],
 ['Coming',
  'WMBBTV',
  '6',
  'A',
  'yard',
  'sale',
  'held',
  'Franklin',
  'County',
  'benefit',
  'family',
  'whose',
  'daughters',
  'died'],
 ['MarshaBlackburn',
  'Since',
  'talking',
  'think',
  'I',
  'think',
  'commit',
  'act',
  'sedition',
  'like',
  'say',
  'taking'],
 ['ScottLinnen',
  'Stonekettle',
  'Where',
  'Im',
  'theres',
  'laws',
  'regarding',
  'displaying',
  'candidate',
  'signage',
  'election',
  'day',
  'E'],
 ['Scott',
  'Morrisons',
  'political',
  'career',
  'At',
  'upcoming',
  'federal',
  'election',
  'Australians',
  'lost',
  'bu']]

Data Preprocessing

In [15]:
import pandas as pd

unique_words = {word for l in all_tweets_no_urls_no_stopwords for word in l}
df_unique_word = pd.DataFrame(unique_words, columns = ["word"])

df_unique_word['id'] = range(1, len(df_unique_word.index) + 1)
df_unique_word.head(5)
df_unique_word_2 = df_unique_word.set_index('word')
word2id = df_unique_word_2.to_dict()

def tweet2ids(tweet):
    ids = [word2id['id'].get(word, '') for word in tweet]
    return ids

transactions = [tweet2ids(tweet) for tweet in all_tweets_no_urls_no_stopwords]

with open("input.txt", 'w') as f:
  for i in transactions:
    f.write(" ".join(str(k) for k in i) + '\n')


In [16]:
import pandas as pd

# PreProcessing
df = pd.DataFrame.from_records(all_tweets_no_urls_no_stopwords)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,FaceTheNation,margbrennan,LindseyGrahamSC,Youve,sunk,low,give,guy,platform,He,thrall,None,None,None,None,None,None,None,None,None,None,None,None,None
1,Coming,WMBBTV,6,A,yard,sale,held,Franklin,County,benefit,family,whose,daughters,died,None,None,None,None,None,None,None,None,None,None
2,MarshaBlackburn,Since,talking,think,I,think,commit,act,sedition,like,say,taking,None,None,None,None,None,None,None,None,None,None,None,None
3,ScottLinnen,Stonekettle,Where,Im,theres,laws,regarding,displaying,candidate,signage,election,day,E,None,None,None,None,None,None,None,None,None,None,None
4,Scott,Morrisons,political,career,At,upcoming,federal,election,Australians,lost,bu,None,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,There,absolutely,evidence,Election,FraudIn,Georgia,242,traffickers,made,5662,trips,ba,None,None,None,None,None,None,None,None,None,None,None,None
996,morethanmySLE,Im,immeasurably,thankful,past,election,hes,far,president,Im,truly,thankful,None,None,None,None,None,None,None,None,None,None,None,None
997,MarshaBlackburn,Arrest,warrants,issued,every,Republican,voted,overturn,results,valid,None,None,None,None,None,None,None,None,None,None,None,None,None,None
998,WowMSNBC,really,going,tubes,This,completely,missing,reality,happening,vot,None,None,None,None,None,None,None,None,None,None,None,None,None,None
